Model architectures
- simple (encoder: lstm, decoder: lstm -> dense)
- stacked_encoder (encoder: lstm -> lstm, decoder: lstm -> dense)
- bistacked_encoder (encoder: bilstm -> lstm, decoder: lstm -> dense)
- stacked_decoder (encoder: lstm, decoder: lstm -> lstm -> dense)
- stacked (encoder: lstm -> lstm, decoder: lstm -> lstm -> dense)
- bistacked (encoder: bilstm -> lstm, decoder: lstm -> lstm -> dense)

### Google Colab utils

In [ ]:
#!pip install keras-tuner

In [ ]:
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [ ]:
#!kill -9 -1

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd "/gdrive/My Drive/air-pollution"

### Modeling

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Reshape, Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.optimizers import Adam

from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

import numpy as np
import pandas as pd
from scipy.ndimage.interpolation import shift
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

In [ ]:
X_train = np.load('./data/third-order/X_train.npy')
y_train = np.load('./data/third-order/y_train.npy')
X_valid = np.load('./data/third-order/X_valid.npy')
y_valid = np.load('./data/third-order/y_valid.npy')
X_test = np.load('./data/third-order/X_test.npy')
y_test = np.load('./data/third-order/y_test.npy')

In [ ]:
Tx, encoder_input_dim = X_train.shape[1], X_train.shape[2]
Ty = y_train.shape[1]
# we feed back only the target variable we are predicting
decoder_input_dim = decoder_output_dim = 1  

In [ ]:
train_encoder_input_data = X_train.copy()
train_decoder_target_data = y_train.copy()
train_decoder_input_data = shift(train_decoder_target_data[:, :, 0].reshape(
                                y_train.shape[0], y_train.shape[1], decoder_input_dim), 
                           shift=[0, 1, 0], cval=-10)

valid_encoder_input_data = X_valid.copy()
valid_decoder_target_data = y_valid.copy()
valid_decoder_input_data = shift(valid_decoder_target_data[:, :, 0].reshape(
                                y_valid.shape[0], y_valid.shape[1], decoder_input_dim), 
                           shift=[0, 1, 0], cval=-10)

In [ ]:
def masked_mse(y_true, y_pred):
    return K.sum(((y_true[:, :, 0] - y_pred[:, :, 0]) ** 2) * (1-y_true[:, :, 1]), 
                  axis=-1) / (1 + K.sum((1-y_true[:, :, 1]), axis=-1))

## Simple seq2seq

In [ ]:
class SimpleSeq2Seq(HyperModel):

    def __init__(self, Tx, Ty, encoder_input_dim, 
                 decoder_input_dim, decoder_output_dim):
        self.Tx = Tx
        self.Ty = Ty
        self.encoder_input_dim = encoder_input_dim
        self.decoder_input_dim = decoder_input_dim
        self.decoder_output_dim = decoder_output_dim

        
    def build(self, hp):

        # ------------------- SHARED LAYERS ---------------------
        latent_dim = hp.Int('latent_dim', min_value=16, max_value=64, step=16)
        
        encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
        decoder_lstm = LSTM(latent_dim, return_sequences=True, 
                            return_state=True, name='decoder_lstm')
        dropout = Dropout(rate=hp.Float('dropout', 0, 0.5, 
                                        step=0.1, default=0.5))
        decoder_dense = Dense(self.decoder_output_dim, 
                              activation='linear', name='decoder_dense')
        
        
        # ---------------------- MODEL ------------------------
        # Define the inputs for the encoder
        encoder_inputs = Input(shape=(self.Tx, self.encoder_input_dim), 
                               name='encoder_input')

        # We discard output and keep the states only.
        _, h, c = encoder_lstm(encoder_inputs)

        # Define an input for the decoder
        decoder_inputs = Input(shape=(self.Ty, self.decoder_input_dim), 
                               name='decoder_input')

        # Obtain all the outputs from the decoder (return_sequences = True)
        decoder_outputs, _, _  = decoder_lstm(decoder_inputs, initial_state=[h, c])

        # Apply dropout
        decoder_outputs = dropout(decoder_outputs)

        # Apply dense 
        decoder_outputs = decoder_dense(decoder_outputs)

        model = Model(inputs=[encoder_inputs, decoder_inputs], 
                      outputs=decoder_outputs)
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, 
                                                sampling='log'))
        model.compile(optimizer=optimizer, loss=masked_mse)

        return model
        

In [ ]:
model_builder = SimpleSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=1000,
                     executions_per_trial=1,
                     directory='local-keras-tuner', 
                     project_name='simple')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=64,
             epochs=100,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)])

In [ ]:
tuner.results_summary()

## Stacked encoder

In [ ]:
class StackedEncoderSeq2Seq(HyperModel):

    def __init__(self, Tx, Ty, encoder_input_dim, 
                 decoder_input_dim, decoder_output_dim):
        self.Tx = Tx
        self.Ty = Ty
        self.encoder_input_dim = encoder_input_dim
        self.decoder_input_dim = decoder_input_dim
        self.decoder_output_dim = decoder_output_dim

        
    def build(self, hp):

        # ------------------- SHARED LAYERS ---------------------
        encoder_latent_dim = hp.Int('encoder_latent_dim', min_value=32, 
                                    max_value=128, step=32)
        shared_latent_dim = hp.Int('shared_latent_dim', min_value=32, 
                                   max_value=128, step=32)
        
        encoder_lstm_1 = LSTM(encoder_latent_dim, return_sequences=True, 
                              name='encoder_lstm_1')
        encoder_dropout = Dropout(rate=hp.Float('encoder_dropout', 0, 0.7, 
                                        step=0.1, default=0.5))
        encoder_lstm_2 = LSTM(shared_latent_dim, return_state=True, 
                              name='encoder_lstm_2')
        decoder_lstm = LSTM(shared_latent_dim, return_sequences=True, 
                            return_state=True, name='decoder_lstm')
        decoder_dropout = Dropout(rate=hp.Float('decoder_dropout', 0, 0.7, 
                                step=0.1, default=0.5))
        decoder_dense = Dense(self.decoder_output_dim, 
                              activation='linear', name='decoder_dense')

        
        # ---------------------- MODEL ------------------------
        # Define the inputs for the encoder
        encoder_inputs = Input(shape=(self.Tx, self.encoder_input_dim), 
                               name='encoder_input')
        
        # Obtain the outputs from the first encoder layer
        encoder_out = encoder_lstm_1(encoder_inputs) 
        
        # Pass the outputs through a dropout layer before 
        # feeding them to the next LSTM layer
        encoder_out = encoder_dropout(encoder_out)
        
        # We discard the output and keep the states only.
        _, h, c = encoder_lstm_2(encoder_out)

        # Define an input for the decoder
        decoder_inputs = Input(shape=(self.Ty, self.decoder_input_dim), 
                               name='decoder_input')

        # Obtain all the outputs from the decoder (return_sequences = True)
        decoder_outputs, _, _  = decoder_lstm(decoder_inputs, initial_state=[h, c])

        # Apply dropout
        decoder_outputs = decoder_dropout(decoder_outputs)

        # Apply dense 
        decoder_outputs = decoder_dense(decoder_outputs)

        model = Model(inputs=[encoder_inputs, decoder_inputs], 
                      outputs=decoder_outputs)
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, 
                                                sampling='log'))
        model.compile(optimizer=optimizer, loss=masked_mse)

        return model
        

In [ ]:
model_builder = StackedEncoderSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=1000,
                     executions_per_trial=1,
                     directory='local-keras-tuner', 
                     project_name='stacked-encoder')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=64,
             epochs=100,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)])

## BiStacked encoder

In [ ]:
class BiStackedEncoderSeq2Seq(HyperModel):

    def __init__(self, Tx, Ty, encoder_input_dim, 
                 decoder_input_dim, decoder_output_dim):
        self.Tx = Tx
        self.Ty = Ty
        self.encoder_input_dim = encoder_input_dim
        self.decoder_input_dim = decoder_input_dim
        self.decoder_output_dim = decoder_output_dim

        
    def build(self, hp):

        # ------------------- SHARED LAYERS ---------------------
        encoder_latent_dim = hp.Int('encoder_latent_dim', min_value=32, 
                                    max_value=128, step=32)
        shared_latent_dim = hp.Int('shared_latent_dim', min_value=32, 
                                   max_value=128, step=32)
        
        encoder_lstm_1 = Bidirectional(LSTM(encoder_latent_dim, return_sequences=True, 
                              name='encoder_lstm_1'))
        encoder_dropout = Dropout(rate=hp.Float('encoder_dropout', 0, 0.7, 
                                        step=0.1, default=0.5))
        encoder_lstm_2 = LSTM(shared_latent_dim, return_state=True, 
                              name='encoder_lstm_2')
        decoder_lstm = LSTM(shared_latent_dim, return_sequences=True, 
                            return_state=True, name='decoder_lstm')
        decoder_dropout = Dropout(rate=hp.Float('decoder_dropout', 0, 0.7, 
                                step=0.1, default=0.5))
        decoder_dense = Dense(self.decoder_output_dim, 
                              activation='linear', name='decoder_dense')

        
        # ---------------------- MODEL ------------------------
        # Define the inputs for the encoder
        encoder_inputs = Input(shape=(self.Tx, self.encoder_input_dim), 
                               name='encoder_input')
        
        # Obtain the outputs from the first encoder layer
        encoder_out = encoder_lstm_1(encoder_inputs) 
        
        # Pass the outputs through a dropout layer before 
        # feeding them to the next LSTM layer
        encoder_out = encoder_dropout(encoder_out)
        
        # We discard the output and keep the states only.
        _, h, c = encoder_lstm_2(encoder_out)

        # Define an input for the decoder
        decoder_inputs = Input(shape=(self.Ty, self.decoder_input_dim), 
                               name='decoder_input')

        # Obtain all the outputs from the decoder (return_sequences = True)
        decoder_outputs, _, _  = decoder_lstm(decoder_inputs, initial_state=[h, c])

        # Apply dropout
        decoder_outputs = decoder_dropout(decoder_outputs)

        # Apply dense 
        decoder_outputs = decoder_dense(decoder_outputs)

        model = Model(inputs=[encoder_inputs, decoder_inputs], 
                      outputs=decoder_outputs)
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, 
                                                sampling='log'))
        model.compile(optimizer=optimizer, loss=masked_mse)

        return model
        

In [ ]:
model_builder = BiStackedEncoderSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=1000,
                     executions_per_trial=1,
                     directory='local-keras-tuner', 
                     project_name='bistacked-encoder')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=64,
             epochs=100,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)])

In [ ]:
tuner.results_summary()

## Stacked decoder

In [ ]:
class StackedDecoderSeq2Seq(HyperModel):

    def __init__(self, Tx, Ty, encoder_input_dim, 
                 decoder_input_dim, decoder_output_dim):
        self.Tx = Tx
        self.Ty = Ty
        self.encoder_input_dim = encoder_input_dim
        self.decoder_input_dim = decoder_input_dim
        self.decoder_output_dim = decoder_output_dim

        
    def build(self, hp):

        # ------------------- SHARED LAYERS ---------------------
        shared_latent_dim = hp.Int('shared_latent_dim', min_value=16, 
                                   max_value=64, step=16)
        decoder_latent_dim = hp.Int('decoder_latent_dim', min_value=16, 
                                    max_value=64, step=16)
        
        encoder_lstm = LSTM(shared_latent_dim, return_state=True, 
                              name='encoder_lstm')
        decoder_lstm_1 = LSTM(shared_latent_dim, return_sequences=True, 
                              name='decoder_lstm_1')
        decoder_dropout_1 = Dropout(rate=hp.Float('decoder_dropout_1', 0, 0.7, 
                                step=0.1, default=0.5))
        decoder_lstm_2 = LSTM(decoder_latent_dim, return_sequences=True, 
                            return_state=True, name='decoder_lstm_2')
        decoder_dropout_2 = Dropout(rate=hp.Float('decoder_dropout_2', 0, 0.7, 
                                step=0.1, default=0.5))
        decoder_dense = Dense(self.decoder_output_dim, 
                              activation='linear', name='decoder_dense')

        
        # ---------------------- MODEL ------------------------
        # Define the inputs for the encoder
        encoder_inputs = Input(shape=(self.Tx, self.encoder_input_dim), 
                               name='encoder_input')
        
        # We discard the output and keep the states only.
        _, h, c = encoder_lstm(encoder_inputs)

        # Define an input for the decoder
        decoder_inputs = Input(shape=(self.Ty, self.decoder_input_dim), 
                               name='decoder_input')

        # Obtain all the outputs from the decoder (return_sequences = True)
        decoder_outputs = decoder_lstm_1(decoder_inputs, initial_state=[h, c])

        # Apply dropout
        decoder_outputs = decoder_dropout_1(decoder_outputs)

        # Apply LSTM again (stacked)
        decoder_outputs, _, _ = decoder_lstm_2(decoder_outputs)
        
        # Apply dropout
        decoder_outputs = decoder_dropout_2(decoder_outputs)
        
        # Apply dense 
        decoder_outputs = decoder_dense(decoder_outputs)

        model = Model(inputs=[encoder_inputs, decoder_inputs], 
                      outputs=decoder_outputs)
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, 
                                                sampling='log'))
        model.compile(optimizer=optimizer, loss=masked_mse)

        return model
        

In [ ]:
model_builder = StackedDecoderSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=1000,
                     executions_per_trial=1,
                     directory='local-keras-tuner', 
                     project_name='stacked-decoder')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=64,
             epochs=100,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)])

In [ ]:
tuner.results_summary()

## Stacked

In [ ]:
class StackedSeq2Seq(HyperModel):

    def __init__(self, Tx, Ty, encoder_input_dim, 
                 decoder_input_dim, decoder_output_dim):
        self.Tx = Tx
        self.Ty = Ty
        self.encoder_input_dim = encoder_input_dim
        self.decoder_input_dim = decoder_input_dim
        self.decoder_output_dim = decoder_output_dim

        
    def build(self, hp):

        # ------------------- SHARED LAYERS ---------------------
        latent_dim = hp.Int('latent_dim', min_value=16, max_value=64, 
                            step=16)
        
        encoder_lstm_1 = LSTM(latent_dim, return_sequences=True,
                              name='encoder_lstm_1')
        encoder_lstm_2 = LSTM(latent_dim, return_state=True, 
                              name='encoder_lstm_2')
        decoder_lstm_1 = LSTM(latent_dim, return_sequences=True, 
                              name='decoder_lstm_1')
        decoder_lstm_2 = LSTM(latent_dim, return_sequences=True, 
                              return_state=True, name='decoder_lstm_2')
        decoder_dense = Dense(self.decoder_output_dim, 
                              activation='linear', name='decoder_dense')
        seq_dropout = Dropout(rate=hp.Float('seq_dropout', 0, 0.7, 
                                step=0.1, default=0.5))
        dense_dropout = Dropout(rate=hp.Float('dense_dropout', 0, 0.7, 
                                step=0.1, default=0.5))

        
        # ---------------------- MODEL ------------------------
        # Define the inputs for the encoder
        encoder_inputs = Input(shape=(self.Tx, self.encoder_input_dim), 
                               name='encoder_input')
        
        # First layer in the encoder
        encoder_outputs = encoder_lstm_1(encoder_inputs)
        
        # Apply dropout
        encoder_outputs = seq_dropout(encoder_outputs)
        
        # Pass the outputs to the next encoder layer, obtain h and c
        _, h, c = encoder_lstm_2(encoder_outputs)

        # Define an input for the decoder
        decoder_inputs = Input(shape=(self.Ty, self.decoder_input_dim), 
                               name='decoder_input')

        # Obtain all the outputs from the decoder (return_sequences = True)
        decoder_outputs = decoder_lstm_1(decoder_inputs, initial_state=[h, c])

        # Apply dropout
        decoder_outputs = seq_dropout(decoder_outputs)

        # Apply LSTM again (stacked)
        decoder_outputs, _, _ = decoder_lstm_2(decoder_outputs)
        
        # Apply dropout
        decoder_outputs = seq_dropout(decoder_outputs)
        
        # Apply dense 
        decoder_outputs = dense_dropout(decoder_outputs)

        model = Model(inputs=[encoder_inputs, decoder_inputs], 
                      outputs=decoder_outputs)
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, 
                                                sampling='log'))
        model.compile(optimizer=optimizer, loss=masked_mse)

        return model
        

In [ ]:
model_builder = StackedSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=1000,
                     executions_per_trial=1,
                     directory='local-keras-tuner', 
                     project_name='stacked')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=64,
             epochs=100,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)])

In [ ]:
drive.flush_and_unmount()

## BiStacked

In [ ]:
class BiStackedSeq2Seq(HyperModel):

    def __init__(self, Tx, Ty, encoder_input_dim, 
                 decoder_input_dim, decoder_output_dim):
        self.Tx = Tx
        self.Ty = Ty
        self.encoder_input_dim = encoder_input_dim
        self.decoder_input_dim = decoder_input_dim
        self.decoder_output_dim = decoder_output_dim

        
    def build(self, hp):

        # ------------------- SHARED LAYERS ---------------------
        latent_dim = hp.Int('latent_dim', min_value=16, max_value=64, 
                            step=16)
        
        encoder_lstm_1 = Bidirectional(LSTM(latent_dim, return_sequences=True,
                              name='encoder_lstm_1'))
        encoder_lstm_2 = LSTM(latent_dim, return_state=True, 
                              name='encoder_lstm_2')
        decoder_lstm_1 = LSTM(latent_dim, return_sequences=True, 
                              name='decoder_lstm_1')
        decoder_lstm_2 = LSTM(latent_dim, return_sequences=True, 
                              return_state=True, name='decoder_lstm_2')
        decoder_dense = Dense(self.decoder_output_dim, 
                              activation='linear', name='decoder_dense')
        seq_dropout = Dropout(rate=hp.Float('seq_dropout', 0, 0.7, 
                                step=0.1, default=0.5))
        dense_dropout = Dropout(rate=hp.Float('dense_dropout', 0, 0.7, 
                                step=0.1, default=0.5))

        
        # ---------------------- MODEL ------------------------
        # Define the inputs for the encoder
        encoder_inputs = Input(shape=(self.Tx, self.encoder_input_dim), 
                               name='encoder_input')
        
        # First layer in the encoder
        encoder_outputs = encoder_lstm_1(encoder_inputs)
        
        # Apply dropout
        encoder_outputs = seq_dropout(encoder_outputs)
        
        # Pass the outputs to the next encoder layer, obtain h and c
        _, h, c = encoder_lstm_2(encoder_outputs)

        # Define an input for the decoder
        decoder_inputs = Input(shape=(self.Ty, self.decoder_input_dim), 
                               name='decoder_input')

        # Obtain all the outputs from the decoder (return_sequences = True)
        decoder_outputs = decoder_lstm_1(decoder_inputs, initial_state=[h, c])

        # Apply dropout
        decoder_outputs = seq_dropout(decoder_outputs)

        # Apply LSTM again (stacked)
        decoder_outputs, _, _ = decoder_lstm_2(decoder_outputs)
        
        # Apply dropout
        decoder_outputs = seq_dropout(decoder_outputs)
        
        # Apply dense 
        decoder_outputs = dense_dropout(decoder_outputs)

        model = Model(inputs=[encoder_inputs, decoder_inputs], 
                      outputs=decoder_outputs)
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, 
                                                sampling='log'))
        model.compile(optimizer=optimizer, loss=masked_mse)

        return model
        

In [ ]:
model_builder = BiStackedSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=1000,
                     executions_per_trial=1,
                     directory='local-keras-tuner', 
                     project_name='bistacked')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=64,
             epochs=100,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)])